In [33]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [34]:
from sailor import RouteGenConfig, SailorDataWarehouse

context="ERP admin panel"

_config = RouteGenConfig.from_env()
_warehouse = SailorDataWarehouse(_config, context, db_path="../db")

routes = await _warehouse.create_routes(count=20)
sessions_group = await _warehouse.create_sessions(routes, count=200)

In [35]:
from typing import List
from sklearn.model_selection import train_test_split
from sailor.types.route_specs import SessionSpec

train_sessions: List[SessionSpec] = []
test_sessions: List[SessionSpec] = []

for _sessions in sessions_group:
    if len(_sessions) <= 1:
        train_sessions.extend(_train)
        continue

    _train, _test = train_test_split(_sessions, shuffle=True, test_size=0.2, random_state=14)
    train_sessions.extend(_train)
    test_sessions.extend(_test)

print(f"Train sessions: {len(train_sessions)}")
print(f"Test sessions: {len(test_sessions)}")

Train sessions: 3200
Test sessions: 800


In [36]:
from sailor.sailor_engine import KNNSailorEngine, SVCSailorEngine

knn_engine = KNNSailorEngine()
knn_engine.fit(routes, train_sessions)

svc_engine = SVCSailorEngine()
svc_engine.fit(routes, train_sessions)

Pipeline(steps=[('tfidf', TfidfVectorizer(stop_words='english')),
                ('svc', LinearSVC(class_weight='balanced', max_iter=2000))])

In [37]:
from abc import ABC, abstractmethod
from typing import Tuple
import pandas as pd
from sailor.sailor_engine import SailorEngine
from sklearn.metrics import classification_report, top_k_accuracy_score

_top_k_score = 5
class ModelValidator(ABC):
    def __init__(self, model: SailorEngine, sessions: List[SessionSpec]):
        self.model = model
        self._test_queries = [s.context for s in sessions]
        self._targets = model.documentor.transform([s.target for s in sessions])
        self._target_names = [
            model.documentor.get_route(id).path
            for id in model.documentor.labels_
        ]

    @abstractmethod
    def predict(self) -> Tuple: ...

    def test_predict(self, query: str):
        routes = self.model.predict(query)[:_top_k_score]

        print(f"\n=== Test query ===")
        print(f"Query: {query}")
        for route in routes:
            print(f"Path: {route.path}, Score: {route.score:.2f}")
            print(f"Context: {route.context}")

    def validate(self):
        predictions, pred_scores = self.predict()
        route_validations = []

        report = classification_report(
            self._targets, predictions,
            target_names=self._target_names,
            zero_division=0,
            output_dict=True)

        for route_id, route in self.model.documentor._routes.items():
            assert isinstance(report, dict)
            metrics = report.get(route.path, {})

            all_tokens = route.context.split()
            unique_tokens = len(set(all_tokens))
            unique_ratio = unique_tokens / len(all_tokens)

            route_validations.append({
                "path": route.path,
                "tokens": len(all_tokens),
                "unique_tokens": unique_tokens,
                "unique_ratio": round(unique_ratio, 2),
                "f1-score": round(metrics.get("f1-score", 0), 2),
                "precision": round(metrics.get("precision", 0), 2),
                "recall": round(metrics.get("recall", 0), 2),
            })

        df = pd.DataFrame(route_validations)

        print(f"\n=== {self.model.__class__.__name__} Analysis ===")
        display(df)
        print(f"Total routes: {len(df)}")
        print(f"Average tokens per route: {df['tokens'].mean():.1f}")
        print(f"Average unique tokens per route: {df['unique_tokens'].mean():.1f}")
        print(f"Average unique ratio: {df['unique_ratio'].mean():.2f}")
        print(f"Average F1 score: {df['f1-score'].mean():.2f}")
        print(f"Average precision: {df['precision'].mean():.2f}")
        print(f"Average recall: {df['recall'].mean():.2f}")

        _top_k_accuracy = top_k_accuracy_score(self._targets, pred_scores, k=_top_k_score)
        print(f"Top-{_top_k_score} accuracy: {_top_k_accuracy:.2f}")

        correlation_metrics = [
            ("unique_ratio", "precision"),
            ("tokens", "recall"),
            ("unique_tokens", "f1-score")
        ]

        print("\n=== Correlations between tokens and metrics ===")
        for metric1, metric2 in correlation_metrics:
            corr = df[metric1].corr(df[metric2])
            print(f"Correlation {metric1} vs {metric2}: {corr:.3f}")

        return df.sort_values("f1-score", ascending=False)

In [38]:
class _KNNModelValidator(ModelValidator):
    def __init__(self):
        super().__init__(knn_engine, test_sessions)

    def predict(self) -> Tuple:
        predictions = self.model.pipeline.predict(self._test_queries)
        pred_proba = self.model.pipeline.predict_proba(self._test_queries)
        return predictions, pred_proba

_knn_engine = _KNNModelValidator()
_knn_engine.validate()

_knn_engine.test_predict("Create a new user in the system")


=== KNNSailorEngine Analysis ===


,path,tokens,unique_tokens,unique_ratio,f1-score,precision,recall
0,/admin/dashboard,242,69,0.29,0.91,0.90,0.93
1,/admin/users,283,96,0.34,0.58,0.76,0.47
2,/admin/users/create,318,59,0.19,0.89,0.94,0.85
3,/admin/users/:id/edit,420,188,0.45,0.69,0.55,0.93
4,/admin/products,235,105,0.45,0.58,0.82,0.45
5,/admin/products/create,283,53,0.19,0.83,0.77,0.90
6,/admin/products/:id/edit,349,89,0.26,0.72,0.56,1.00
7,/admin/orders,262,115,0.44,0.85,0.97,0.75
8,/admin/orders/:id,241,23,0.10,0.88,0.78,1.00
9,/admin/customers,207,82,0.40,0.53,0.88,0.38


Total routes: 20
Average tokens per route: 283.1
Average unique tokens per route: 95.8
Average unique ratio: 0.34
Average F1 score: 0.76
Average precision: 0.79
Average recall: 0.76
Top-5 accuracy: 0.97

=== Correlations between tokens and metrics ===
Correlation unique_ratio vs precision: 0.163
Correlation tokens vs recall: 0.411
Correlation unique_tokens vs f1-score: -0.260

=== Test query ===
Query: Create a new user in the system
Path: /admin/users/create, Score: 0.31
Context: admin users create users create erp admin new account add user Setup new user Creating user account employee onboarding Add new accoutn ERP system Create new user account default settings New user creation accounting department Adding new user system New user setup Create new user account admin panel Add new user account warehouse team New user set create user New user creation Add new user system Create user admin Add new user admin panel New user creation add users different roles Creating new user specific

In [39]:
class _SVCModelValidator(ModelValidator):
    def __init__(self):
        super().__init__(svc_engine, test_sessions)

    def predict(self) -> Tuple:
        predictions = self.model.pipeline.predict(self._test_queries)
        _decisions = self.model.pipeline.decision_function(self._test_queries)
        return predictions, _decisions

_svc_engine = _SVCModelValidator()
_svc_engine.validate()

_svc_engine.test_predict("Create a new user in the system")


=== SVCSailorEngine Analysis ===


,path,tokens,unique_tokens,unique_ratio,f1-score,precision,recall
0,/admin/dashboard,242,69,0.29,0.93,0.90,0.95
1,/admin/users,283,96,0.34,0.56,0.75,0.45
2,/admin/users/create,318,59,0.19,0.86,0.87,0.85
3,/admin/users/:id/edit,420,188,0.45,0.71,0.58,0.93
4,/admin/products,235,105,0.45,0.54,0.74,0.42
5,/admin/products/create,283,53,0.19,0.85,0.79,0.93
6,/admin/products/:id/edit,349,89,0.26,0.73,0.58,0.97
7,/admin/orders,262,115,0.44,0.85,0.97,0.75
8,/admin/orders/:id,241,23,0.10,0.88,0.78,1.00
9,/admin/customers,207,82,0.40,0.50,0.88,0.35


Total routes: 20
Average tokens per route: 283.1
Average unique tokens per route: 95.8
Average unique ratio: 0.34
Average F1 score: 0.75
Average precision: 0.78
Average recall: 0.76
Top-5 accuracy: 0.96

=== Correlations between tokens and metrics ===
Correlation unique_ratio vs precision: 0.139
Correlation tokens vs recall: 0.358
Correlation unique_tokens vs f1-score: -0.209

=== Test query ===
Query: Create a new user in the system
Path: /admin/users/create, Score: -0.20
Context: admin users create users create erp admin new account add user Setup new user Creating user account employee onboarding Add new accoutn ERP system Create new user account default settings New user creation accounting department Adding new user system New user setup Create new user account admin panel Add new user account warehouse team New user set create user New user creation Add new user system Create user admin Add new user admin panel New user creation add users different roles Creating new user specifi

In [40]:
_models_path = "../build/models"
knn_engine_path = knn_engine.save_model("knn_model", _models_path)
knn_model_size_mb = os.path.getsize(knn_engine_path)
print(f"Model saved to {knn_engine_path}, size: {knn_model_size_mb} MB")

svc_engine_path = svc_engine.save_model("svc_model", _models_path)
svc_model_size_mb = os.path.getsize(svc_engine_path)
print(f"Model saved to {svc_engine_path}, size: {svc_model_size_mb} MB")

Model saved to ../build/models/knn_model.pkl, size: 451162525 MB
Model saved to ../build/models/svc_model.pkl, size: 451300912 MB
